<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word2vect


In [1]:
import numpy as np

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [16]:
def get_vocabulary(corpus: np.ndarray) -> np.ndarray:
    voc_set = set()
    doc_trimmed_in_words = np.char.split(corpus)

    for element in doc_trimmed_in_words:
        voc_set = voc_set.union(set(element))

    voc_list = list(voc_set)
    voc_list.sort()
    
    return np.array(voc_list)

vocabulary = get_vocabulary(corpus=corpus)
print(vocabulary)
print(len(vocabulary))

['de' 'dia' 'el' 'es' 'gracias' 'hoy' 'martes' 'muchas' 'que']
9


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [24]:
voc_dict = {value: count for count, value in enumerate(vocabulary)}
voc_dict

{'de': 0,
 'dia': 1,
 'el': 2,
 'es': 3,
 'gracias': 4,
 'hoy': 5,
 'martes': 6,
 'muchas': 7,
 'que': 8}

In [33]:
def ohe_np(voc_dict: dict, corpus: np.ndarray) -> np.ndarray:
    n_classes = len(voc_dict)
    n_documents = len(corpus)

    result = np.zeros((n_documents, n_classes))

    for index, document in enumerate(corpus):
        row_result = result[index]
        
        for word in document.split():
            row_result[voc_dict[word]] = 1

    return result

ohe_np(voc_dict, corpus)

array([[0., 1., 0., 1., 0., 1., 0., 0., 1.],
       [1., 1., 1., 1., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1., 0.]])

### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [34]:
def frec_np(voc_dict: dict, corpus: np.ndarray) -> np.ndarray:
    n_classes = len(voc_dict)
    n_documents = len(corpus)

    result = np.zeros((n_documents, n_classes))

    for index, document in enumerate(corpus):
        row_result = result[index]
        
        for word in document.split():
            row_result[voc_dict[word]] += 1

    return result

frec_np(voc_dict, corpus)

array([[0., 1., 0., 1., 0., 1., 0., 0., 1.],
       [1., 1., 1., 1., 0., 1., 2., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1., 0.]])

### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [47]:
def tfidf_np(voc_dict: dict, corpus: np.ndarray) -> np.ndarray:
    n_classes = len(voc_dict)
    n_documents = len(corpus)

    ohe_matrix = ohe_np(voc_dict, corpus)
    df_array = np.sum(ohe_matrix, axis=0)
    frec_matrix = frec_np(voc_dict, corpus)

    result = np.zeros((n_documents, n_classes))

    for index, document in enumerate(corpus):
        row_result = result[index]

        for j, word in enumerate(document.split()):
            tf = frec_matrix[index, voc_dict[word]]
            idf = np.log10(n_documents/df_array[voc_dict[word]])

            row_result[voc_dict[word]] = tf * idf

    return result

tfidf_np(voc_dict, corpus)

array([[0.        , 0.17609126, 0.        , 0.17609126, 0.        ,
        0.17609126, 0.        , 0.        , 0.47712125],
       [0.47712125, 0.17609126, 0.47712125, 0.17609126, 0.        ,
        0.17609126, 0.35218252, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.47712125,
        0.        , 0.17609126, 0.47712125, 0.        ]])

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [61]:
def cosine_similarity_by_doc_in_corpus(corpus: np.ndarray, index: int) -> np.ndarray:
    document = corpus[index]
    similarity_array = np.zeros((len(corpus)))

    tfidf_matrix = tfidf_np(voc_dict, corpus)

    for i, doc in enumerate(corpus):
        similarity = cosine_similarity(tfidf_matrix[index], tfidf_matrix[i])
        similarity_array[i] = similarity

    #print(similarity_array)

    indices = np.argsort(similarity_array)[::-1][:len(corpus)]

    return corpus[indices]

print(corpus)
print(cosine_similarity_by_doc_in_corpus(corpus, 0))
print(cosine_similarity_by_doc_in_corpus(corpus, 1))
print(cosine_similarity_by_doc_in_corpus(corpus, 2))

['que dia es hoy' 'martes el dia de hoy es martes' 'martes muchas gracias']
['que dia es hoy' 'martes el dia de hoy es martes' 'martes muchas gracias']
['martes el dia de hoy es martes' 'que dia es hoy' 'martes muchas gracias']
['martes muchas gracias' 'martes el dia de hoy es martes' 'que dia es hoy']
